# Cleaning players data using SQLite

In the last notebook 'Grabbing_all_player_stats-June12', we stored the box score (specifically players stats) for almost every NBA game in 2004-2005 season. With about 36,000 games since 2004, this resulted in about 480,000 player records collected, with 24 fields collected per record.

In collecting this data, there were issues with a small proportion of games: either with '--' stored instead of a stat or another critical error. We will first examine how many games had these types of errors. We will then clean our data by handling records with these types of errors.

We are interested in cleaning the player data from the 2008-2009 season to the 2017-2018 season. There are two main reasons we restrict to these seasons for cleaning player data:

1. In the notebook "Scraping_all_team_stats- June11", we gathered and organized team stats for all the games during these 10 seasons.

2. Teams didn't keep track of the stat plus/minus before the 2008-2009 season, at least on ESPN.com. For a player, plus/minus is the difference between how much his team scores and how much his opponent scores while the player is in the game. We will see later that even some of the plus/minus stats that are available cannot be trusted.

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns',None)
from pandas.util.testing import assert_frame_equal


import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
from nose.tools import assert_equal

import sqlite3


## Cleaning 2017-2018 season

We will start by cleaning players stats data from the 2017-2018 season. In the previous notebook, we found issues with three games during this season, with Matchup ID's 400975115, 400975606, and 400975750 and boxscores found at http://www.espn.com/nba/boxscore?gameId=400975115, http://www.espn.com/nba/boxscore?gameId=400975606, and http://www.espn.com/nba/boxscore?gameId=400975750, respectively. Examining the box scores, we find that each boxscore has at least one player record with stats listed as '--' or '-----'. As there is no information offered by these records, we will simply eliminate them.

In [2]:
#write 2017-2018 players stats to DataFrame

#convert all columns to type string (some were originally int)
#remove unnamed column to have indices being row number in DataFrame
players_stats_18 = pd.read_csv('all_players_stats_2018.csv', dtype=str).loc[:,'team_name':]

In [3]:
players_stats_18.head()

,team_name,player_name,player_id,position,started,played,min,fg_made,fg_attempted,3pt_made,3pt_attempted,ft_made,ft_attempted,oreb,dreb,reb,ast,stl,blk,to,pf,plusminus,pts,matchup_id
0,OKC,carmelo-anthony,1975,PF,yes,yes,32,3,6,3,3,0,0,1,2,3,2,2,1,0,2,12,9,400975872
1,OKC,corey-brewer,3191,SF,yes,yes,29,5,8,1,4,3,4,1,1,2,3,2,0,1,4,6,14,400975872
2,OKC,paul-george,4251,SF,yes,yes,39,9,20,3,6,5,5,0,7,7,6,4,0,3,3,5,26,400975872
3,OKC,steven-adams,2991235,C,yes,yes,37,5,11,0,0,0,2,5,8,13,1,0,0,2,3,10,10,400975872
4,OKC,russell-westbrook,3468,PG,yes,yes,38,7,19,0,4,5,8,1,10,11,5,3,1,7,5,7,19,400975872


In [4]:
#check that stats are now type string
assert_equal(players_stats_18.loc[0,'fg_made'], '3')
assert_equal(type(players_stats_18.loc[0,'fg_attempted']), str)
assert_equal(type(players_stats_18.loc[0,'matchup_id']), str)

We will check that we have players stats information for every game during the 2017-2018 season. To do this, we will consider the game information we previously stored for the 2017-2018 season (written to the CSV file 'all_games_04_on.csv' in the notebook 'Accumulating all games since 2004-2005 season-June11'). We will then compare the number of Matchup ID's we stored there to the number of games that are represented in our players stats file.

In [5]:
#info on every game since 2004-2005 season
all_game_info = pd.read_csv('all_games_04_on.csv')

#info on every game in 2017-2018 season
all_game_info_18 = all_game_info[all_game_info['season_end_year']==2018].loc[:,'team':]

all_game_info_18.head()

,team,season_start_year,season_end_year,season_type,game_month,game_day,game_year,game_date,matchup_id
31336,bos,2017,2018,regular,10,17,2017,10/17/2017,400974437
31337,bos,2017,2018,regular,10,18,2017,10/18/2017,400974703
31338,bos,2017,2018,regular,10,20,2017,10/20/2017,400974772
31339,bos,2017,2018,regular,10,24,2017,10/24/2017,400974802
31340,bos,2017,2018,regular,10,26,2017,10/26/2017,400974818


In [6]:
#number of games represented in all games information file
print('Number of 2017-2018 games in all games file: ' + str(len(set(all_game_info_18.loc[:,'matchup_id']))))

#number of games represented in players stats file
print('Number of 2017-2018 games in players stats file: ' + str(len(set(players_stats_18.loc[:,'matchup_id']))))

Number of 2017-2018 games in all games file: 1312
Number of 2017-2018 games in players stats file: 1312


They agree! This means that we were able to successfully pull the box score from every game (regular and post season) during the 2017-2018 season. 

Before saving save this table, we change the column names of the table so that they are compatible with SQL commands. Specifically, we change `min` to `minutes`, `3pt_made` to `three_pt_made`, and `3pt_attempted` to `three_pt_attempted`.

In [7]:
print(players_stats_18.columns.tolist())

['team_name', 'player_name', 'player_id', 'position', 'started', 'played', 'min', 'fg_made', 'fg_attempted', '3pt_made', '3pt_attempted', 'ft_made', 'ft_attempted', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'to', 'pf', 'plusminus', 'pts', 'matchup_id']


In [8]:
column_names = players_stats_18.columns.tolist()

column_names[column_names.index('min')] = 'minutes'
column_names[column_names.index('3pt_made')] = 'three_pt_made'
column_names[column_names.index('3pt_attempted')] = 'three_pt_attempted'

players_stats_18.columns = column_names

print(players_stats_18.columns.tolist())

print(players_stats_18.shape)

players_stats_18.head()

['team_name', 'player_name', 'player_id', 'position', 'started', 'played', 'minutes', 'fg_made', 'fg_attempted', 'three_pt_made', 'three_pt_attempted', 'ft_made', 'ft_attempted', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'to', 'pf', 'plusminus', 'pts', 'matchup_id']
(33100, 24)


,team_name,player_name,player_id,position,started,played,minutes,fg_made,fg_attempted,three_pt_made,three_pt_attempted,ft_made,ft_attempted,oreb,dreb,reb,ast,stl,blk,to,pf,plusminus,pts,matchup_id
0,OKC,carmelo-anthony,1975,PF,yes,yes,32,3,6,3,3,0,0,1,2,3,2,2,1,0,2,12,9,400975872
1,OKC,corey-brewer,3191,SF,yes,yes,29,5,8,1,4,3,4,1,1,2,3,2,0,1,4,6,14,400975872
2,OKC,paul-george,4251,SF,yes,yes,39,9,20,3,6,5,5,0,7,7,6,4,0,3,3,5,26,400975872
3,OKC,steven-adams,2991235,C,yes,yes,37,5,11,0,0,0,2,5,8,13,1,0,0,2,3,10,10,400975872
4,OKC,russell-westbrook,3468,PG,yes,yes,38,7,19,0,4,5,8,1,10,11,5,3,1,7,5,7,19,400975872


In [9]:
players_stats_18.to_csv('all_players_stats_2018_string.csv')

## Using SQLite to clean 2017-2018 data

We want to view which player records have '--' for stats and then remove these records. Rather than persisting to use pandas DataFrames, we will switch to using SQLite databases. Not only will this allow to show the use of SQL commands, the types of queries we will be interested in will easier to write using SQL. 

For this, I will be using what I learned while taking a yearlong Data Science course taught by Professor Robert Brunner at the University of Illinois. His website can be found at https://astro.illinois.edu/directory/profile/bigdog.

In [10]:
# First find our HOME directory
home_dir = !echo $HOME

#Define data directory
database_dir = home_dir[0] + '/database'

print(f'Database will persist at {database_dir}\n')

Database will persist at /Users/derekjung/database



In [11]:
%%bash -s "$database_dir"

#passed Python variable, later accessed with $1

#check if directory exists
if [ -d "$1" ] ; then

    echo "Directory already exists."

else
    #otherwise grapb file from Internet and store locally in data directory
    
    mkdir $1
    echo "creating database directory"

fi

Directory already exists.


In [12]:
con = sqlite3.connect("stats_617.db")

cur = con.cursor()

In [13]:
#DataFrame with all entries type string

players_stats_18.to_sql(name='all_players_stats_2018_string', con=con, if_exists='replace', \
                        index=False, chunksize=1000)

In [14]:
#check that we can access 2018 player stats
sql_access = "\
SELECT * \
FROM all_players_stats_2018_string \
LIMIT 3 \
"

cur.execute(sql_access)

for row in cur:
    print(row)

('OKC', 'carmelo-anthony', '1975', 'PF', 'yes', 'yes', '32', '3', '6', '3', '3', '0', '0', '1', '2', '3', '2', '2', '1', '0', '2', '12', '9', '400975872')
('OKC', 'corey-brewer', '3191', 'SF', 'yes', 'yes', '29', '5', '8', '1', '4', '3', '4', '1', '1', '2', '3', '2', '0', '1', '4', '6', '14', '400975872')
('OKC', 'paul-george', '4251', 'SF', 'yes', 'yes', '39', '9', '20', '3', '6', '5', '5', '0', '7', '7', '6', '4', '0', '3', '3', '5', '26', '400975872')


We now find the four player records from the 2017-2018 season that led to errors due to the presence of "--". As mentioned earlier, these rows come from the games with Matchup ID's 400975115, 400975606, and 400975750.

In [15]:
#find rows with errors
sql_errors_2018 = "\
SELECT * \
FROM all_players_stats_2018_string \
WHERE minutes = '--'\
"

cur.execute(sql_errors_2018)

for row in cur:
    print(row)

('PHX', 'derrick-jones-jr', '3936099', 'SG', 'no', 'yes', '--', None, None, None, None, None, None, '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '400975115')
('SAC', 'dimitrios-agravanis', '3899663', 'PF', 'no', 'yes', '--', None, None, None, None, None, None, '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '400975606')
('HOU', 'troy-williams', '3078286', 'SF', 'no', 'yes', '--', None, None, None, None, None, None, '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '400975606')
('POR', 'brandon-rush', '3457', 'SG', 'no', 'yes', '--', None, None, None, None, None, None, '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '400975750')


We choose to delete these four rows as they offer no new information about player performance. To check that we have actually deleted four rows, we will show the number of rows before and after the `DELETE` statement.

In [16]:
#find table length before deleting
sql_length_before_delete_2018 = "SELECT COUNT(*) FROM all_players_stats_2018_string"

cur.execute(sql_length_before_delete_2018)

print("Table length before deletion: ", end='')

for row in cur.fetchone():
    print(row)

#delete rows with '--'
sql_delete_2018 = "DELETE FROM all_players_stats_2018_string WHERE minutes = '--'"

cur.execute(sql_delete_2018)

print('Deleting... Deleting...')


#find table length after deleting
sql_length_after_delete_2018 = "SELECT COUNT(*) FROM all_players_stats_2018_string"

cur.execute(sql_length_after_delete_2018)

print("Table length after deletion: ", end='')

for row in cur.fetchone():
    print(row)

Table length before deletion: 33100
Deleting... Deleting...
Table length after deletion: 33096


We now save the table to a DataFrame and convert each stat column to type `int`.

In [17]:
players_stats_2018_cleaned = pd.read_sql("SELECT * FROM all_players_stats_2018_string;", con)

minutes_index = 6

#columns to convert to type int
#choose not to convert Matchup ID column (otherwise column would be converted to type int64)
columns_to_int = players_stats_2018_cleaned.columns.tolist()[6:-1] 

print(columns_to_int)

for column in columns_to_int: 
    players_stats_2018_cleaned.loc[:,column] = players_stats_2018_cleaned.loc[:,column].apply(lambda x: int(x))


print(players_stats_2018_cleaned.shape)
players_stats_2018_cleaned.head()

['minutes', 'fg_made', 'fg_attempted', 'three_pt_made', 'three_pt_attempted', 'ft_made', 'ft_attempted', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'to', 'pf', 'plusminus', 'pts']
(33096, 24)


,team_name,player_name,player_id,position,started,played,minutes,fg_made,fg_attempted,three_pt_made,three_pt_attempted,ft_made,ft_attempted,oreb,dreb,reb,ast,stl,blk,to,pf,plusminus,pts,matchup_id
0,OKC,carmelo-anthony,1975,PF,yes,yes,32,3,6,3,3,0,0,1,2,3,2,2,1,0,2,12,9,400975872
1,OKC,corey-brewer,3191,SF,yes,yes,29,5,8,1,4,3,4,1,1,2,3,2,0,1,4,6,14,400975872
2,OKC,paul-george,4251,SF,yes,yes,39,9,20,3,6,5,5,0,7,7,6,4,0,3,3,5,26,400975872
3,OKC,steven-adams,2991235,C,yes,yes,37,5,11,0,0,0,2,5,8,13,1,0,0,2,3,10,10,400975872
4,OKC,russell-westbrook,3468,PG,yes,yes,38,7,19,0,4,5,8,1,10,11,5,3,1,7,5,7,19,400975872


In [18]:
#curiously columns are being converted to type numpy.int64

for column in columns_to_int:
    print(column + ' type: ' + str(type(players_stats_2018_cleaned.loc[0,column])))

minutes type: <class 'numpy.int64'>
fg_made type: <class 'numpy.int64'>
fg_attempted type: <class 'numpy.int64'>
three_pt_made type: <class 'numpy.int64'>
three_pt_attempted type: <class 'numpy.int64'>
ft_made type: <class 'numpy.int64'>
ft_attempted type: <class 'numpy.int64'>
oreb type: <class 'numpy.int64'>
dreb type: <class 'numpy.int64'>
reb type: <class 'numpy.int64'>
ast type: <class 'numpy.int64'>
stl type: <class 'numpy.int64'>
blk type: <class 'numpy.int64'>
to type: <class 'numpy.int64'>
pf type: <class 'numpy.int64'>
plusminus type: <class 'numpy.int64'>
pts type: <class 'numpy.int64'>


In [19]:
#save cleaned 2017-2018 players stats to CSV file

players_stats_2018_cleaned.to_csv('players_stats_2018_cleaned.csv')

## Cleaning 2013-2014 to 2016-2017 player stats

We will now move on to cleaning the next four seasons: 2013-2014, 2014-2015, 2015-2016, and 2016-2017. By looking at the error log saved in the text file 'player_stats_errors_2017_2014', it appears that the same type of error occurs for these season as in the first season. Specifically, some player stats for games are erroneously filled with '--' or '-----'. 

In [20]:
#print the first 30 lines of the error file
with open('player_stats_errors_2017_2014.txt', 'r') as fout:
    counter = 0
    for line in fout:
        print(line.split())
        counter += 1
        if counter > 30:
            break

['Start', 'of', '2017', 'season.']
['400900168', 'min']
['400900168', 'fg_made']
['400900168', 'fg_attempted']
['400900168', '3pt_made']
['400900168', '3pt_attempted']
['400900168', 'ft_made']
['400900168', 'ft_attempted']
['400900168', 'oreb']
['400900168', 'dreb']
['400900168', 'reb']
['400900168', 'ast']
['400900168', 'stl']
['400900168', 'blk']
['400900168', 'to']
['400900168', 'pf']
['400900168', 'plusminus']
['400900168', 'pts']
['400900276', 'min']
['400900276', 'fg_made']
['400900276', 'fg_attempted']
['400900276', '3pt_made']
['400900276', '3pt_attempted']
['400900276', 'ft_made']
['400900276', 'ft_attempted']
['400900276', 'oreb']
['400900276', 'dreb']
['400900276', 'reb']
['400900276', 'ast']
['400900276', 'stl']
['400900276', 'blk']


We see that the error messages are exactly the lists of length 2 when split by spaces. We store all of the Matchup ID's in a list.

In [21]:
with open('player_stats_errors_2017_2014.txt', 'r') as fout:
    ids_with_errors_2017_2014 = []
    for line in fout:
        if len(line.split()) == 2:
            ids_with_errors_2017_2014.append(line.split()[0])

We make the claim that for a game if one column has an issue, then all columns have the issue. To check this, we will show that each erroneous matchup id occurs 17 times (the total number of stats). Note that the maximum number of times each matchup id could occur is 17.

In [22]:
ids_with_errors_2017_2014_set = set(ids_with_errors_2017_2014)

for matchup_id in ids_with_errors_2017_2014_set:
    assert_equal(ids_with_errors_2017_2014.count(matchup_id), 17)

We will now write a function that returns the cleaned players stats table for a given year.

In [23]:
def clean_dash_rows(year, conn, curs):
    '''
    Cleans a DataFrame by removing rows filled with dashes removed.
    
    Input:
    year: int between 2005 and 2018
    conn: connection
    curs: cursor
    
    Output:
    DataFrame
    '''
    
    #import players stats from year
    data_path = 'all_players_stats_' + str(year) + '.csv'
    players_stats_df = pd.read_csv(data_path, dtype=str).loc[:,'team_name':]
    
    players_stats_df_copy = players_stats_df.copy()
    
    #change column names to be compatible with SQL
    column_names = players_stats_df_copy.columns.tolist()

    column_names[column_names.index('min')] = 'minutes'
    column_names[column_names.index('3pt_made')] = 'three_pt_made'
    column_names[column_names.index('3pt_attempted')] = 'three_pt_attempted'

    players_stats_df_copy.columns = column_names
    
    #convert players stats DataFrame to SQL table
    players_stats_df_copy.to_sql(name='players_stats_df_string', con=conn, if_exists='replace', \
                        index=False, chunksize=1000)
    
    #delete rows with '--'
    sql_delete = "DELETE FROM players_stats_df_string WHERE minutes = '--'"

    curs.execute(sql_delete)
    
    #convert shorter table to DataFrame
    players_stats_df_cleaned = pd.read_sql("SELECT * FROM players_stats_df_string;", conn)
    
    
    #convert columns to type numpy int64
    
    minutes_index = 6

    #columns to convert to type int
    #choose not to convert Matchup ID column (otherwise column would be converted to type int64)
    columns_to_int = players_stats_df_cleaned.columns.tolist()[6:-1] 

    for column in columns_to_int: 
        players_stats_df_cleaned.loc[:,column] = players_stats_df_cleaned.loc[:,column].apply(lambda x: int(x))

    return players_stats_df_cleaned

In [24]:
#import players stats from year
data_path = 'all_players_stats_' + str(2018) + '.csv'
players_stats_df = pd.read_csv(data_path, dtype=str).loc[:,'team_name':]

#check that function works for 2017-2018 stats
assert_frame_equal(clean_dash_rows(2018,con,cur), players_stats_2018_cleaned)

It works! We now write these DataFrames to files, from the 2013-2014 season to the 2016-2017 season.

In [25]:
base_cleaned_df_to_csv = 'players_stats_{0}_cleaned.csv'

#initial problem with CSV files stored in cloud instead of on computer
for year in range(2014,2018):
    clean_dash_rows(year,con,cur).to_csv(base_cleaned_df_to_csv.format(year))

## Cleaning 2012-2013 player data

For the 2012-2013 season, there were problems with the 4 games of Matchup ID's 400277756, 400277874, 400278328, and 400278344. Three of the errors are By looking at their boxscores at http://www.espn.com/nba/boxscore?gameId=400277756, http://www.espn.com/nba/boxscore?gameId=400277874, http://www.espn.com/nba/boxscore?gameId=400278328, http://www.espn.com/nba/boxscore?gameId=400278344, all except one of the box scores suffer from the same problem: one player's stats has dashes in every spot. The exception is a November match between the Raptors and 76ers, where a player's name is mistakenly written as `null` and no actual player is attached to it. After seeing how many games total there are during the season, we will simply choose to ignore the stats from this game.

In [26]:
#contains all game info from games during 2012-2013 season
all_game_info_2013 = all_game_info[all_game_info['season_end_year']==2013]

print('Number of games during 2012-2013 season: ' + \
     str(len(set(all_game_info_2013.loc[:,'matchup_id']))))

print('Percentage of one game of whole season: ' \
      + str(100*1/len(set(all_game_info_2013.loc[:,'matchup_id']))) + '%')

Number of games during 2012-2013 season: 1312
Percentage of one game of whole season: 0.07621951219512195%


In [27]:
#remove rows from DataFrame that are actual players followed by dashes
clean_dash_rows(2013,con,cur).to_csv(base_cleaned_df_to_csv.format(2013))

## Cleaning 2004-2005 to 2011-2012 player data

We will now focus on cleaning the rest of the player data. Our eventual goal will be to simply ignore games that lead to critical errors, i.e., games for which there other errors than just actual players with dashes for stats. Last notebook, we saved the log of errors in a text file called 'player_stats_errors_2012_2004'.

We will keep track of the errors with two dictionaries: `dash_errors` and `other_errors`. The keys will be the years from 2005 to 2012. The values of `dash_errors` will be all of the matchup ID's in that year with the issue that an actual player has _some_ number of stats that cannot be converted into `int`'s, e.g., if stats are listed as dashes. The values of `other_errors` will be the matchup ID's in that year with some other type of error in extraction.

We begin by dividing up our error log by year.

In [28]:
#make list of errors from 2004-2012
with open('player_stats_errors_2012_2004.txt','r') as fout:
    all_years_errors = [error.strip() for error in fout]

In [29]:
print(all_years_errors[:5])

['Start of 2012 season.', '320211030 min', '320211030 fg_made', '320211030 fg_attempted', '320211030 3pt_made']


In [30]:
all_years_errors.index('Start of 2011 season.') #410
print(all_years_errors[407:412])

['320217018 plusminus', '320217018 pts', '2012 took 1392.1917550563812 seconds.', 'Start of 2011 season.', '310223003 min']


In [31]:
#show other error message
for idx, error in enumerate(all_years_errors):
    if 'Problem' in error:
        print(idx)
        print(error)
        break

496
Problem with 310428001


There is a pattern for the error log of each season. A season will begin with 'Start of {year} season.' season spanned over {year-1} and {year}. It will then list errors of the form '{Matchup ID} {name of stat}' or 'Problem with {Matchup ID}'. It will conclude with a message saying how long that season took. 

We break up our long error log by year, starting at 2012 and ending at 2005.

In [32]:
#key: year (from 2012 to 2005)
#value: index of item in all_years_errors that is 'Start of {year} season.'
start_season_dict = {}

year = 2012

for idx, error in enumerate(all_years_errors):
    if 'Start' in error: #beginning of season
        start_season_dict[year] = idx
        year -= 1
    if year == 2004: #last year included in errors
        break

In [33]:
#check that start_season_dict has years from 2012 to 2005 as keys
assert_equal(sorted(list(start_season_dict.keys())),list(range(2005,2013)))

#check that indices correctly align with starts of season for each year
for year in range(2012,2004,-1):
    assert_equal(all_years_errors[start_season_dict[year]],'Start of {0} season.'.format(year))

In [34]:
#break up list of errors by year
errors_by_years = {}

for year in range(2006,2013):
    errors_by_years[year] = all_years_errors[start_season_dict[year]:start_season_dict[year-1]]
    
errors_by_years[2005] = all_years_errors[start_season_dict[2005]:]

In [35]:
assert_equal(errors_by_years[2007][0], 'Start of 2007 season.')
assert_equal('took' in errors_by_years[2007][-1], True) #should be '2007 took {time} seconds.'

We now create and fill out the dash_errors and other_errors dictionary. The key difference between the structure of the two error messages is that a dash error message has length 2 (when split by spaces) while other errors have length 3.

In [36]:

#keys of dictionaries
years = [year for year in range(2005,2013)]
#values are lists
empty_lists = [[] for year in range(2005,2013)]

dash_errors = {}#dict(zip(years,empty_lists))
other_errors = {}#dict(zip(years,empty_lists))

for year in range(2005, 2013):
    for error in errors_by_years[year]:
        #other error
        if len(error.split())==3:
            if year in other_errors:
                other_errors[year].append(error.split()[2])
            else:
                other_errors[year] = [error.split()[2]]
            #other_errors[year].append(error.split()[2]) #store Matchup ID in other errors dictionary
            #print('-'*50 + 'yo its another other error')
            
        #other error
        elif len(error.split())==2:
            if year in dash_errors:
                dash_errors[year].append(error.split()[0])
            else:
                dash_errors[year] = [error.split()[0]]
            #dash_errors[year].append(error.split()[0]) #store Matchup ID in dash error dictionary
            #print('dash error')

In [37]:
#check that dictionaries stored some correct Matchup ID's for 2010
year_2010 = 2010

for error in errors_by_years[year_2010]:
    if len(error.split()) == 2:
        assert_equal(error.split()[0] in dash_errors[year_2010], True)
        break

for error in errors_by_years[year_2010]:
    if len(error.split()) == 3:
        assert_equal(error.split()[2] in other_errors[year_2010], True)

## Checking dash errors 

For the first 6 seasons that we cleaned, we were fortunate if a single game stat for a player was dashed, i.e., a player's stat was listed as '--' or '-----', then all of the player's game stats were dashed. This might not hold true in future seasons. For example, during the 2007-2008 season, 'plusminus' (which for a player, equals the number of points the player's team scored minus the points the opponent team scored while the player was win) was not kept track of on ESPN.com during the 2007-2008 season and before. See, for instance, the box score at http://www.espn.com/nba/boxscore?gameId=271103026 for a November 2007 game between the Warriors and the Jazz.

We are interested in the lists dash_errors[year], where year ranges between 2009 and 2018. Note that if a player has all dashes for stats, then the game will appear 17 times (the total number of stats) in the dash errors list. We are interested if all dash errors occur this way. We will analyze this by counting the number of times each game appears in the dash errors list. For this, we will use `Counter` from the `collections` module.

In [38]:
#print(dash_errors[2012])
print(other_errors[2011])

['310428001', '310512001', '310426019', '310510004', '310424001', '310508001', '310422001', '310506001', '310420024', '310504004', '310502004', '310412013', '310406024', '310429029', '310427024', '310425029', '310423029', '310419019', '310417024', '310413030', '310413021', '310327029', '310411001', '310409024', '310416019', '310405001', '310403024', '310401010']


In [39]:
from collections import Counter

#keys: year
#values: Counter objects that keep track of how many stats for games have dash errors
dash_errors_counts = {}
other_errors_counts = {}
for year in range(2009, 2013):
    try:
        dash_errors_counts[year] = Counter(dash_errors[year])
    except KeyError as error:
        print('{0}: No dash errors'.format(year))
    try:
        other_errors_counts[year] = Counter(other_errors[year])
    except KeyError as error:
        print('{0}: No other errors'.format(year))

2012: No other errors


Oh wonderful! All dash errors occur with all stats being dashes. We will now check that this holds true down to the 2008-2009 season.

In [41]:
#keeps track of games where less 17 stats affected
dash_not_max = 0

for year in range(2009, 2012):
    for matchup_id in dash_errors_counts[year]:
        if dash_errors_counts[year][matchup_id] != 17:
            print('Y: {0}, ID: {1}, Count: {2}'.format(year, matchup_id, dash_errors_counts[year][matchup_id]))
            dash_not_max += 1

print('Number of games with 1-17 dash errors: {0}'.format(dash_not_max))

Number of games with 1-17 dash errors: 0


It does hold true! We will simply remove rows with dashes for the 'minutes' stat, since those are exactly the rows with dashes for any stats, which coincide with the rows that have dashes for all stats.

Just for kicks, we will see how many dash errors occurred by season.

In [42]:
for year in range(2009,2013):
    print('Year {0}: {1} dash error rows'.format(year, len(dash_errors_counts[year])))

Year 2009: 37 dash error rows
Year 2010: 14 dash error rows
Year 2011: 18 dash error rows
Year 2012: 24 dash error rows


In [43]:
#remove rows with dash errors
for year in range(2009,2013):
    clean_dash_rows(year,con,cur).to_csv(base_cleaned_df_to_csv.format(year))

To conclude this notebook, we will determine how many games had a critical error other than dashes. We will also examine some of the box scores of games with these errors. Recall we already found that the 2011-2012 season had no other types of errors.

In [44]:
for year in range(2009, 2013):
    #print number of other errors by year
    try:
        print('{0}: {1} other critical errors'.format(year, len(other_errors_counts[year])))
    
    #no other errors for that year
    except KeyError as error:
        print('{0}: 0 other critical errors'.format(year))

2009: 15 other critical errors
2010: 43 other critical errors
2011: 28 other critical errors
2012: 0 other critical errors


Let's calculate the percentage of games that have other critical errors by year. For this, we will use SQL to first find the number of unique Matchup ID's per year.

In [45]:
#keys: years between 2009 and 2012
#values: set of Matchup ID's for that year
num_games = {}

#for year in range(2009, 2013):
 #   all_matchup_ids = all_game_info[all_game_info['season_end_year']==year].loc[:,'matchup_id']


In [46]:
all_game_info.loc[:,'team':].to_sql(name='all_games_info', con=con, if_exists='replace', \
                        index=False, chunksize=1000)

#count number of games by nmber of Matchup ID's
sql_num_games = "\
SELECT season_end_year, COUNT(DISTINCT matchup_id) \
FROM all_games_info \
WHERE season_end_year <= 2012 \
AND season_end_year >= 2009 \
GROUP BY season_end_year \
"

cur.execute(sql_num_games)

#keys: years between 2009 and 2012
#values: set of Matchup ID's for that year
num_games = {}

for row in cur:
    num_games[row[0]] = row[1]
    
print(num_games) #2011-2012 season was strike-shortened

{2009: 1315, 2010: 1312, 2011: 1311, 2012: 1074}


In [47]:
#percentage of games with critical error by year
print('Percentage of games with another critical error: ')
for year in range(2009,2012):
    print('{0}: {1:.2f}%'.format(year, 100 * len(other_errors_counts[year])/num_games[year]))
print('2012: 0%')

Percentage of games with another critical error: 
2009: 1.14%
2010: 3.28%
2011: 2.14%
2012: 0%


There are very few games (relatively) that have critical errors other than dash errors. Recall how we dealt with games with other types of critical errors when retrieving their player stats: we simply didn't retrieve the box scores. Thus, these games are completely omitted from the list of box scores. We choose to simply keep it this way rather than trying ad hoc methods to retrieve their player stats.

In [48]:
#close cursor and connection since we no longer will do SQL commands
cur.close()
con.close()

We conclude by viewing some of the box scores of error games during the 2010 season.

In [49]:
#retreive Matchup ID's for 5 games with other critical errors
error_games_2010 = list(other_errors_counts[2010].keys())[:5]

error_games_2010

['300331008', '300331024', '300329017', '300411018', '300325004']

In [50]:
print('Corresponding box scores at links')
print('-'*50)

base_box_score_url = 'http://www.espn.com/nba/boxscore?gameId={0}'

for matchup_id in error_games_2010:
    print(base_box_score_url.format(matchup_id))

Corresponding box scores at links
--------------------------------------------------
http://www.espn.com/nba/boxscore?gameId=300331008
http://www.espn.com/nba/boxscore?gameId=300331024
http://www.espn.com/nba/boxscore?gameId=300329017
http://www.espn.com/nba/boxscore?gameId=300411018
http://www.espn.com/nba/boxscore?gameId=300325004


Examining the box scores, it isn't clear what is wrong them that led to the critical error. 

However, we do make one observation. In all of the games, there are unreasonable values for plus/minus. More specifically, there are players in each game with plus/minus larger than +7 or less than -7 who played 0 minutes. This is nearly impossible, and wouldn't occur with such frequency. (It's very, very rare that a team scores more than 10 points in a single minute.) Thus, we will need to take our plus/minus stats with a grain of salt moving forward. 